In [1]:
import pandas as pd
import numpy as np
import pprint as pp
import re, string
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import praw

import credentials as creds

In [2]:
# https://praw.readthedocs.io/en/latest/getting_started/
# https://www.reddit.com/dev/api/

In [3]:
CLIENT_ID = creds.client_id()
CLIENT_SECRET_KEY = creds.client_secret_key()


r = praw.Reddit(client_id = CLIENT_ID,
                client_secret = CLIENT_SECRET_KEY,
                user_agent = 'RedditorMatch')

print(r.read_only)

True


In [4]:
def getThreads(subreddit, limit = 20):
    thread_ids = []
    
    hotThreads = subreddit.hot(limit = limit)
    time.sleep(2)
    topThreads = subreddit.top(limit = limit)
    time.sleep(2)
    newThreads = subreddit.new(limit = limit)    
    
    for t in hotThreads:
        if (t.id not in thread_ids):
            thread_ids.append(t.id)
        
    for t in topThreads:
        if (t.id not in thread_ids):
            thread_ids.append(t.id)
        
    for t in newThreads:
        if (t.id not in thread_ids):
            thread_ids.append(t.id)
    
    return(thread_ids)

In [5]:
def getUsersFromSubreddit(subredditIds):
    if(len(subredditIds) < 1):
        return None
    
    usernames = []
    
    ## this process takes a while...    

    for thread_id in subredditIds:        
        thread = r.submission(id = thread_id)
        time.sleep(2)
        
        try:
            username = thread.author.name # gets the author of the thread 
            if (username is not None and username not in usernames):
                print("--- Adding user:", username)
                usernames.append(username)
        except:
            print("------ Unable to get thread author. Moving on...")      

        # get all authors of the comments in the thread
        allComments = thread.comments.list()
        for c in allComments:
            try:
                username = str(c.author)
                if (username is not None and username not in usernames):
                    print("--- Adding user:", username)
                    usernames.append(username)
            except:
                print("------ failed. moving on...")
                
        time.sleep(2)
                
    return(usernames)      

In [6]:
def cleanText(text):
    text = text.replace(',', ' ').replace(':', ' ')\
                .replace('...', ' ')\
                .replace('?', ' ')\
                .replace('!', ' ')\
                .replace(';', ' ')\
                .replace('\n', ' ').replace('\r', '') # replace newlines and page breaks     
                
    text = re.sub(r'([^\s\w]|_)+', '', text) # remove non-alphanumeric characters but leave the spaces
    text = re.sub(' +',' ', text) # remove double spaces        
    return(text)

In [7]:
def getRedditor(username):
    try:     
        username = username.strip()
        user = r.redditor(username)
    except Exception as e: 
        print(e)
        return None
    
    return(user)

In [8]:
def getUserComments(username, commentsLimit = 1000):   
    userComments = []
    

    counter = 0
    while (counter <= 3):
        user = getRedditor(username)
        if (user is not None):
            break
        else:
            counter = counter + 1               
            print("------------ Retry #", counter, "for user:", username)
            time.sleep(15)
        
    if (user is None):
        print("------ ERROR: unable to retrieve user info:", username)
        return None  
    
    time.sleep(2)
    
    try:
        newComments = user.comments.new(limit = commentsLimit)
    except Exception as e: 
        print(e)
        print("------ ERROR: unable to retrieve new comments. moving on...")
        newComments = None
    
    time.sleep(2)
    
    try:
        hotComments = user.comments.hot(limit = commentsLimit)
    except Exception as e: 
        print(e)
        print("------ ERROR: unable to retrieve hot comments. moving on...")
        hotComments = None
    
    time.sleep(2)
    
    try:
        controversialComments = user.comments.controversial(limit = commentsLimit)
    except Exception as e: 
        print(e)
        print("------ ERROR: unable to retrieve controversial comments. moving on...")
        controversialComments = None

    # new comments    
    try:
        if (newComments is not None):
            for c in newComments:
                comment = cleanText(c.body)        
                if (comment not in userComments):
                    userComments.append(comment)
    except:
        print("Failed...moving on...")
    
    # hot comments
    try:
        if (hotComments is not None):
            for c in hotComments:  
                comment = cleanText(c.body)        
                if (comment not in userComments):
                    userComments.append(comment)
    except:
        print("Failed...moving on...")

    # controversial comments
    try:
        if (controversialComments is not None):
            for c in controversialComments:
                comment = cleanText(c.body)     
                if (comment not in userComments):
                    userComments.append(comment)
    except:
        print("Failed...moving on...")
        
    print("------ Retrieved", len(userComments), "comments for:", username)
    return(userComments)

In [9]:
def subExists(subreddit):
    exists = True
    try:
        r.subreddits.search_by_name(subreddit, exact = True)
    except:
        exists = False
    return(exists)

In [10]:
def scrapeCommentsFromSubreddit(subreddit):
    
    UserArray = []
    CommentArray = []  
    
    exists = subExists(subreddit)
    
    if (exists == False):
        print("ERROR: subreddit '", subreddit, "' does not exist...")
        return None
    
    sub = r.subreddit(subreddit)
    
    threads = getThreads(sub)
    
    if (len(threads) < 1):
        print("ERROR: unable to retrieve threads for subreddit:", subreddit)
    else:
        print("Retrieved ", len(threads), "threads...")
    
    users = getUsersFromSubreddit(threads)
    
    if (len(users) < 1):
        print("ERROR: unable to retrieve users for subreddit:", subreddit)
    else:
        print("Retrieved ", len(users), "users...")
    
    for user in users:
        try:
            userComments = getUserComments(user)

            if (userComments is not None):
                userComments = ".".join(userComments)
                UserArray.append(user)
                CommentArray.append(userComments)
        except:
            print("ERROR: something failed badly. moving on...")
        
    df = pd.DataFrame(UserArray, columns = ["Username"])
    df["Comments"] = CommentArray
    
    print("Outputting dataframe with ", len(df), "records")
    
    return(df)

In [11]:
#mizzou = scrapeCommentsFromSubreddit("mizzou")

#mizzou.to_csv("data/mizzou.csv", index = False)

In [12]:
#umich = scrapeCommentsFromSubreddit("uofm")

#umich.to_csv("data/umich.csv", index = False)

In [13]:
#chicago = scrapeCommentsFromSubreddit("chicago")

#chicago.to_csv("data/chicago.csv", index = False)

In [ ]:
#columbia = scrapeCommentsFromSubreddit("columbiamo")

#columbia.to_csv("data/columbia.csv", index = False)

In [ ]:
dems = scrapeCommentsFromSubreddit("democrats")

dems.to_csv("data/dems.csv", index = False)

Retrieved  43 threads...
--- Adding user: data2dave
--- Adding user: LovingWestVirginia
--- Adding user: biznash
--- Adding user: Kevzzhere
--- Adding user: alvarezg
--- Adding user: gunnergoz
--- Adding user: socrates_scrotum
--- Adding user: election_info_bot
--- Adding user: poldim
--- Adding user: arachnivore
--- Adding user: DingJones
--- Adding user: SliderUp
--- Adding user: Bricknody
--- Adding user: praxulus
--- Adding user: decatur8r
--- Adding user: supermanbluegoldfish
--- Adding user: Rakajj
--- Adding user: Shivshanks
--- Adding user: randomaccessvagaries
--- Adding user: None
--- Adding user: interested21
--- Adding user: The_Write_Stuff
--- Adding user: Plisskens_snake
--- Adding user: Bomaruto
--- Adding user: uneducated_scientist
--- Adding user: artisanrox
--- Adding user: DarkTechnocrat
--- Adding user: israeljeff
--- Adding user: Professor_Skywalker
--- Adding user: sscilli
--- Adding user: j-awesome
--- Adding user: dolphins3
--- Adding user: wwaxwork
--- Adding u

--- Adding user: ajdrausal
--- Adding user: -seibah-
--- Adding user: kygsapycf
--- Adding user: PM_ME_DUCKS
--- Adding user: axechamp75
--- Adding user: TinTinCT617
--- Adding user: iamahotblondeama
--- Adding user: AndThereBeDragons
--- Adding user: pillinprogress
--- Adding user: Arenvan
--- Adding user: askheidi
--- Adding user: Encyphus
--- Adding user: isellem
--- Adding user: BlueNotesBlues
--- Adding user: gumout
--- Adding user: Laminar_flo
--- Adding user: YouAintFromTheWoods
--- Adding user: patrickthag
--- Adding user: VHSCopyOfGoodFellas
--- Adding user: undercover_shill
--- Adding user: kingchilifrito
--- Adding user: LateDentArthurDent42
--- Adding user: Youthsonic
--- Adding user: NugatoryDescripti0n
--- Adding user: Back4smith
--- Adding user: LordNedNoodle
--- Adding user: theonlydidymus
--- Adding user: da_doctori
--- Adding user: maxmcleod
--- Adding user: The-Young-Buffet
--- Adding user: Incog-neeeto
--- Adding user: PoorLittleLamb
--- Adding user: Jwhiz0309
--- A

--- Adding user: Gsteel11
--- Adding user: mayIspeakanonymously
--- Adding user: Literally_Fake_News
--- Adding user: let_me_more
--- Adding user: TrueDeceiver
--- Adding user: RoseL123
--- Adding user: blastzone24
--- Adding user: kotoamatsukamix
--- Adding user: melyop
--- Adding user: Zenith_and_Quasar
--- Adding user: Thestonersteve
--- Adding user: Gnarledhalo
--- Adding user: bodie425
--- Adding user: jsanc623
--- Adding user: spiritualskywalker
--- Adding user: taws34
--- Adding user: nerkidner
--- Adding user: Boukish
--- Adding user: ImpotentRocket
--- Adding user: Sea_of_Blue
--- Adding user: ciras
--- Adding user: because-i-can-
--- Adding user: CunningMan59
--- Adding user: sweetwattah
--- Adding user: Xenotoz
--- Adding user: Sparky400Hz
--- Adding user: bogboom
--- Adding user: Cirri
--- Adding user: jdrc07
--- Adding user: gimpwiz
--- Adding user: a_cups_are_superior
--- Adding user: ASPD_Account
--- Adding user: SwampDrainer
--- Adding user: facehead502
--- Adding user:

--- Adding user: PrestoVivace
--- Adding user: Pud500001
--- Adding user: sAlander4
--- Adding user: damnatio_memoriae
--- Adding user: AnAngryAmerican
--- Adding user: Jocramid
--- Adding user: TransparentIcon
--- Adding user: igits
--- Adding user: ROLLtrumpinTIDE
--- Adding user: PinkSlimeIsPeople
--- Adding user: AnonPenusGuy
--- Adding user: YourShittyGrammar
--- Adding user: CharlieDarwin2
--- Adding user: Aquabrah
--- Adding user: treble_hook
--- Adding user: bokan
--- Adding user: digitalnuisance
--- Adding user: set616
--- Adding user: Exact_bro
--- Adding user: JournalismIsDead
--- Adding user: SuperPartyPooper
--- Adding user: BumwineBaudelaire
--- Adding user: CRISPR
--- Adding user: Koninator
--- Adding user: Parabarabappa
--- Adding user: brssnj45
--- Adding user: fuckemalldead
--- Adding user: FuckTheLord
--- Adding user: bhath01
--- Adding user: unequivocali
--- Adding user: theyrerightbehindyou
--- Adding user: Slyder
--- Adding user: jmerc83
--- Adding user: I_Hate_
-

--- Adding user: dr_isk_16
--- Adding user: Wesandersonisgod
--- Adding user: UWCG
--- Adding user: SPER
--- Adding user: timbenj77
--- Adding user: That1RandomKid
--- Adding user: Wardamntoucan
--- Adding user: ISaidThatOnPurpose
--- Adding user: Datasaurus_Rex
--- Adding user: form_the_turtle
--- Adding user: branchbranchley
--- Adding user: rpm3311
--- Adding user: Wood_floors_are_wood
--- Adding user: ericcris10sen
--- Adding user: ChineseMeatCleaver
--- Adding user: Gludius
--- Adding user: tehnibi
--- Adding user: NotCausarius
--- Adding user: TriggerHappy_NZ
--- Adding user: OldYellerIn4K
--- Adding user: Whoelseisbored
--- Adding user: burnerphonelurker
--- Adding user: Defaiz
--- Adding user: shleedogga1
--- Adding user: Toking_Ginger
--- Adding user: reckufttub
--- Adding user: Fuzzybuddha
--- Adding user: Nathonator
--- Adding user: GlobeTrott3r
--- Adding user: Forotosh
--- Adding user: otterom
--- Adding user: kvist56
--- Adding user: mmat7
--- Adding user: TeeribleMureal


...

------ Retrieved 1169 comments for: Boks1
------ Retrieved 407 comments for: Dankmaymays_XD
------ Retrieved 1512 comments for: SirSeizureSalad
------ Retrieved 1175 comments for: redpilled_brit
------ Retrieved 1702 comments for: Leetmcfeet
------ Retrieved 1253 comments for: Cheeseypoofs123
------ Retrieved 10 comments for: DonkeyOfCongo
------ Retrieved 104 comments for: Toasted_Cheeze
------ Retrieved 813 comments for: NoDrugZone
------ Retrieved 1849 comments for: AMERICANFUNK
------ Retrieved 1688 comments for: Deathstreet
------ Retrieved 292 comments for: TheRedsAreComing
------ Retrieved 909 comments for: uterus_at_capacity
------ Retrieved 1606 comments for: Impeach_Pence
------ Retrieved 1895 comments for: fluffykerfuffle1
------ Retrieved 944 comments for: Lay3rs0Fc0nfusion
------ Retrieved 1128 comments for: CapitalismForFreedom
------ Retrieved 251 comments for: Gruskungen
------ Retrieved 1283 comments for: Not-creepy
------ Retrieved 11 comments for: TurdBurgler5000
---

------ Retrieved 1254 comments for: ExoticIsle
------ Retrieved 560 comments for: account781
------ Retrieved 4 comments for: JamesAndBluee
------ Retrieved 126 comments for: WexDs
------ Retrieved 82 comments for: pvrisyelyah
------ Retrieved 1885 comments for: AreYouDeaf
------ Retrieved 70 comments for: DontKatalaveno
------ Retrieved 486 comments for: TheOpenedMind
------ Retrieved 1785 comments for: noxumida
------ Retrieved 1574 comments for: HairyMongoose
------ Retrieved 1414 comments for: eazyirl
------ Retrieved 57 comments for: VisualizedSuccess
------ Retrieved 448 comments for: grteagrea
------ Retrieved 1903 comments for: JitGoinHam
------ Retrieved 1972 comments for: Anandya
------ Retrieved 1455 comments for: Nearlydearly
Failed...moving on...
Failed...moving on...
Failed...moving on...
------ Retrieved 0 comments for: grandmaangryy
------ Retrieved 1782 comments for: anutensil
------ Retrieved 1029 comments for: milesrhoden
------ Retrieved 1697 comments for: unforgiva

------ Retrieved 1761 comments for: NatakuNox
------ Retrieved 45 comments for: teapotdomeman
------ Retrieved 680 comments for: tomascamus
------ Retrieved 1942 comments for: CriminalMacabre
------ Retrieved 94 comments for: Bptashi
------ Retrieved 1005 comments for: bnelson1
------ Retrieved 1659 comments for: muskegthemoose
------ Retrieved 1947 comments for: Sanhael
------ Retrieved 1316 comments for: Bigpiganddig
------ Retrieved 1900 comments for: Dark_Shroud
------ Retrieved 402 comments for: wellwaffled
------ Retrieved 1257 comments for: Ofbearsandmen
------ Retrieved 855 comments for: xStealthySidearms
------ Retrieved 1438 comments for: BlackCrowRises
------ Retrieved 236 comments for: Status_Quo__
------ Retrieved 305 comments for: kdc1026910
------ Retrieved 962 comments for: razorblazer420
------ Retrieved 919 comments for: Smoda
------ Retrieved 1081 comments for: jakelasersnake
------ Retrieved 1010 comments for: Storyfiend
------ Retrieved 366 comments for: Jofreebs
-

------ Retrieved 748 comments for: Skate_a_book
------ Retrieved 697 comments for: PM_UR_DEAD_HOOKERS
------ Retrieved 226 comments for: Karnage-Kimaris
------ Retrieved 1928 comments for: poppymelt
------ Retrieved 1030 comments for: Ciertocarentin
------ Retrieved 1486 comments for: DankandSpank
------ Retrieved 51 comments for: digidead
------ Retrieved 125 comments for: B-Retawesor
------ Retrieved 510 comments for: DownVotesAreLife
------ Retrieved 990 comments for: Indispute
------ Retrieved 1926 comments for: Stamps_dot_com
------ Retrieved 631 comments for: iruleU
------ Retrieved 116 comments for: Obsessedsole
------ Retrieved 1835 comments for: CompMolNeuro
------ Retrieved 1694 comments for: PmMeYourYeezys
------ Retrieved 149 comments for: Kaneh-bos
------ Retrieved 651 comments for: Chemaz
------ Retrieved 377 comments for: Chickenlover12
------ Retrieved 1890 comments for: cisxuzuul
------ Retrieved 1876 comments for: Humankeg
------ Retrieved 759 comments for: Swarrel
--

------ Retrieved 1291 comments for: W0lfgangster
------ Retrieved 1101 comments for: everyman50
------ Retrieved 1994 comments for: kickstand
------ Retrieved 1594 comments for: droddt
------ Retrieved 1405 comments for: Ebonic_Plague
------ Retrieved 165 comments for: shins530
------ Retrieved 927 comments for: barkologix
------ Retrieved 1437 comments for: Aruno
------ Retrieved 1037 comments for: codereign
------ Retrieved 447 comments for: MrNoSouls
------ Retrieved 1933 comments for: kensal78
------ Retrieved 1970 comments for: warpfield
------ Retrieved 637 comments for: cranbrook_aspie
------ Retrieved 911 comments for: Stebetto
------ Retrieved 1891 comments for: comrade-jim
------ Retrieved 255 comments for: StockPilot24
------ Retrieved 1860 comments for: eb86
------ Retrieved 306 comments for: FederalFarmerHM
------ Retrieved 350 comments for: ThisNameWorks
------ Retrieved 390 comments for: astroturding
------ Retrieved 1853 comments for: YourLovelyMan
------ Retrieved 1819

In [ ]:
reps = scrapeCommentsFromSubreddit("republicans")

reps.to_csv("data/reps.csv", index = False)

In [ ]:
bball = scrapeCommentsFromSubreddit("CollegeBasketball")

bball.to_csv("data/bball.csv", index = False)

In [ ]:
austin = scrapeCommentsFromSubreddit("austin")

austin.to_csv("data/austin.csv", index = False)